In [ ]:
import pyspark as ps

spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("Case study") 
        .getOrCreate()
        )
sc = spark.sparkContext
tweet_path = '../data/french_tweets.json'
rdd_tweet = sc.textFile(tweet_path)
#rdd = spark.read.json('../data/french_tweets.json')
#print(rdd.first())
#print(df.sample(1))
print("Done")

In [ ]:
df = spark.read.json(rdd_tweet)

#import json
#datastore = json.loads(first)
#for key in datastore:
#    print(key, datastore[key])
print("Done")

In [ ]:
first = df.first()
type(first)

In [ ]:
df.printSchema()

In [ ]:
first = df.first()

In [ ]:
sam = df.sample(.05)

In [ ]:
sample_count = sam.count()

In [ ]:
df.createOrReplaceTempView("tweets")
result = spark.sql('''
    SELECT id, text, created_at, retweeted
    FROM tweets
    ORDER BY retweeted DESC
    ''')
result.show()
result.count()

In [ ]:
min_max_dates = spark.sql('''
    SELECT MIN(created_at), MAX(created_at)
    FROM tweets
    ''')
min_max_dates.show()

In [ ]:
macron_mention = result.filter(result["text"].rlike("macron"))
macron_mention.show()
macron_mention.count()/sample_count

In [ ]:
pen_mention = result.filter(result["text"].rlike("lepen"))
#pen_mention.show()
pen_mention.count()/sample_count

In [ ]:
macron_mention = result.filter(result["text"].rlike("macron"))
result.count(), macron_mention.count(), pen_mention.count()

In [ ]:
TOTAL TWEETS, MACRON MENTION, PEN MENTION

In [ ]:
from pyspark.sql.functions import col
result = df.select(col("id"), col("text"), col("entities").hashtags)
#result = result.map(lambda x: x["entities"]["hashtags"])
from collections import defaultdict
hashtag_counts = defaultdict(int)

t = result.take(1000000)
for thing in t:
    hashtag_thingy = thing['entities.hashtags']
    if not hashtag_thingy:
        continue
    for row in hashtag_thingy:
        hashtag_counts[row.text.lower()] += 1
hashtag_counts = dict(hashtag_counts)
hashtag_counters = sorted(hashtag_counts.items(),key=lambda pair: pair[1],reverse=True)
hashtag_counters[:100]

In [ ]:
t = result.collect()
hashtag_counts = {}
for thing in t:
    hashtag_thingy = thing['entities.hashtags']
    if not hashtag_thingy:
        continue
    for row in hashtag_thingy:
        if row.text.lower() not in hashtag_counts:
            hashtag_counts[row.text.lower()] = 1
        else:
            hashtag_counts[row.text.lower()] += 1
hashtag_counters = sorted(hashtag_counts.items(),key=lambda pair: pair[1],reverse=True)
hashtag_counters[:10]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
p_df = pd.DataFrame(hashtag_counters[:10])
p_df[0]
plt.figure(figsize=(20,12))
plt.style.use('ggplot')
plt.bar("#" + p_df[0], p_df[1])
plt.xticks(rotation=60, size = 20)
plt.yticks(size = 20)
plt.title("Top 10 most popular hashtags in our n = 200,000 dataset", size = 30)
plt.ylabel("# Tweets using hashtag", size = 20)
plt.savefig("test.jpg")